# Compare Signal and Volume

### To illustrate changes over treatment, compare the median signal within the prostate and the volume change over treatment

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

df = pd.read_csv('../Output/Submission-Long/Features_Rescaled.csv')
df = df[df['ContourType'] == 'Manual']
df = df[df['Feature'].isin(['shape_MeshVolume', 'firstorder_Median'])]
df = df[['PatID','Fraction','Feature','FeatureValue']]

In [ ]:
df_signal = df[df['Feature'] == 'firstorder_Median']
df_signal = df_signal[['PatID','Fraction','FeatureValue']]
df_signal = df_signal.rename(columns={'FeatureValue': 'Signal'})

df_volume = df[df['Feature'] == 'shape_MeshVolume']
df_volume = df_volume[['PatID','Fraction','FeatureValue']]
df_volume = df_volume.rename(columns={'FeatureValue': 'Volume'})

In [ ]:
df_plot = pd.merge(df_signal, df_volume, on=['PatID','Fraction'])
df_plot = df_plot.melt(id_vars=['PatID','Fraction'], var_name='Feature', value_name='Value')
# rename to Signal and Volume
df_plot['Feature'] = df_plot['Feature'].replace({'Signal': 'MedianSignal', 'Volume': 'Volume'})

In [ ]:
# plot using matplotlib

sns.set_style('whitegrid')
sns.despine()
fig, ax = plt.subplots(figsize=(12, 6))
# custom_params = {"axes.spines.right": False, "axes.spines.top": False}

sns.lineplot(data=df_plot, x='Fraction', y='Value', hue='Feature', ax=ax, ci=95, 
             legend='full', palette='hls', linewidth=2.5, markers=True, style='Feature', estimator=np.mean,
             markersize=10)#, err_style='bars', err_kws={'elinewidth': 1.5, 'capsize': 3, 'capthick': 1.5})
ax.set_xlabel('Fraction', fontsize=20)
ax.set_ylabel('Feature Value', fontsize=20)
ax.set_title('Changes in Median Signal and Volume over Treatment', fontsize=20, pad=20)
ax.legend(loc='upper right', borderaxespad=0.)
ax.set_xlim(0.95, 5.05)
ax.set_xticks([1, 2, 3, 4, 5])
ax.set_xticklabels(['1', '2', '3', '4', '5'], fontsize=16)
ax.set_ylim(0.1, 0.6)
#ax.set_yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6], fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=16)
# pad title, move up
plt.subplots_adjust(top=0.9)
# make legend bigger
plt.setp(ax.get_legend().get_texts(), fontsize='16')
plt.setp(ax.get_legend().get_title(), fontsize='16')
plt.legend(bbox_to_anchor=(0.8, 0.9), loc=2, borderaxespad=0., fontsize=16)


plt.tight_layout()
plt.savefig('./lineplot_signal_volume.png', dpi=300)

plt.show()


In [ ]:
df_signal = df_plot[df_plot['Feature'] == 'MedianSignal']
df_signal = df_signal[['PatID','Fraction','Value']]
df_signal_l = df_signal.copy()
df_signal_d = df_signal.loc[df_signal['Fraction'].isin([1, 5])]

In [ ]:
# get random patients
patIDs = df_signal['PatID'].unique()
rand_pats = np.random.choice(patIDs, 3, replace=False)
print(rand_pats)

In [ ]:
df_signal_d['Model'] = 'Delta'
df_signal_l['Model'] = 'Longitudinal'
df_plt2 = df_signal_d.append(df_signal_l)
df_plt2 = df_plt2[df_plt2['PatID'].isin(rand_pats)]
# sort reverse order bu mode;
df_plt2 = df_plt2.sort_values(by=['Model'], ascending=False)